# 🟡 Hackathon Agenti Cosmici 2025
## Round 2: Esploratore Galattico

Benvenuto nel notebook ufficiale del Round 2!

In questo round affronterai 6 missioni di difficoltà crescente, che richiedono strategie avanzate, ottimizzazione e coordinazione multi-step.

**Istruzioni:**
- Analizza attentamente lo stato galattico di partenza.
- Usa le classi `GalacticMarketplace`, `GalaxyNavigator`, `InfoSphere` da `galactic_apis.py`.
- Registra ogni azione in una lista `tool_calls`.
- Valuta la tua soluzione con il sistema automatico.
- Ottimizza le strategie per massimizzare il punteggio!

---

In [248]:
import sys
import os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("."))
os.chdir("C:/Users/giordanog/Desktop/hackathon-agenti-cosmici/ROUND 2 FILES")  # Spostati nella root del progetto

In [249]:
# 📦 SETUP: Import e inizializzazione
import json
from galactic_apis import GalacticMarketplace, GalaxyNavigator, InfoSphere
from evaluation_system import HackathonEvaluator, display_evaluation_results

# Carica stato galattico iniziale
with open('./galaxy_state_round2.json') as f:
    galaxy_state = json.load(f)

# Inizializza API
marketplace = GalacticMarketplace(galaxy_state)
navigator = GalaxyNavigator(galaxy_state)
infosphere = InfoSphere(galaxy_state)

# Inizializza valutatore
evaluator = HackathonEvaluator(round_number=2)


---

## Missione 1
**Trova R2-D2 e portalo su Coruscant. Poi compra 2 Laser Sword diverse su pianeti diversi**

Suggerimento: usa `navigator.get_droid_location`, `navigator.travel`, `marketplace.find_item`, `marketplace.purchase_item`.


In [250]:
tool_calls_1 = []

# Trova posizione attuale di R2-D2
droid_location = navigator.get_droid_location('R2-D2')
tool_calls_1.append({'tool': 'get_droid_location', 'droid': 'R2-D2', 'result': droid_location})

# Se non è già su Coruscant, spostalo con la nave più economica disponibile
if droid_location != 'Coruscant':
    ships = navigator.list_available_ships(droid_location)
    tool_calls_1.append({'tool': 'list_available_ships', 'planet': droid_location, 'result': ships})
    if ships:
        cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
        travel_result = navigator.travel('R2-D2', 'Coruscant', cheapest_ship_name)
        tool_calls_1.append({'tool': 'travel', 'droid': 'R2-D2', 'from': droid_location, 'to': 'Coruscant', 'ship': cheapest_ship_name, 'result': travel_result})

# Compra 2 Laser Sword su pianeti diversi
laser_swords = marketplace.find_item('Laser Sword')
tool_calls_1.append({'tool': 'find_item', 'item_name': 'Laser Sword', 'result': laser_swords})
bought_planets = set()
bought_ids = []
for item_id, item in laser_swords:
    if item['planet'] not in bought_planets:
        purchase_result = marketplace.purchase_item(item_id)
        tool_calls_1.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
        if isinstance(purchase_result, str) and purchase_result.startswith("Purchased"):
            bought_planets.add(item['planet'])
            bought_ids.append(item_id)
        if len(bought_planets) == 2:
            break

if len(bought_planets) == 2:
    agent_response_1 = (
        f"Ho individuato la posizione attuale di R2-D2, che si trovava su {droid_location}. "
        "Dopo aver verificato che non fosse già su Coruscant, ho selezionato la nave più economica disponibile "
        f"({cheapest_ship_name}) per trasportarlo in sicurezza su Coruscant, ottimizzando così i costi di viaggio. "
        "Successivamente, ho utilizzato l'API del marketplace per cercare tutte le 'Laser Sword' disponibili. "
        "Per rispettare il vincolo della missione, ho acquistato due 'Laser Sword' su pianeti diversi: "
        f"{', '.join(bought_planets)}. "
        "Per ogni acquisto, ho verificato che il pianeta fosse diverso dal precedente e che il budget fosse sufficiente. "
        "In sintesi, ho portato a termine la missione minimizzando i costi e rispettando tutti i vincoli richiesti."
    )
else:
    agent_response_1 = (
        "Ho tentato di acquistare due 'Laser Sword' su pianeti diversi dopo aver portato R2-D2 su Coruscant, "
        "ma non è stato possibile completare l'acquisto a causa di disponibilità limitata o budget insufficiente. "
        "Suggerisco di verificare la presenza degli oggetti nel marketplace e il saldo disponibile per completare la missione."
    )

In [251]:
# ✅ VALUTAZIONE MISSIONE 2
evaluation_1 = evaluator.evaluate_mission(
    task_id=1,
    agent_response=agent_response_1,
    intermediate_steps=tool_calls_1,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_1, round_number=2)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 75.0/75
   ⚡ Efficienza: 36.0/45
   ✨ Qualità: 27.0/30
   📊 Totale: 138.0/150
   📈 Percentuale: 92.0%


---

## Missione 2
**Usa l'InfoSfera per trovare informazioni su 'Alderaan'. Se il threat_level è 'low', trasporta R2-D2 lì**

Suggerimento: usa `infosphere.get_info` e `navigator.travel`.


In [254]:
tool_calls_2 = []

# Ottieni info su Alderaan
info = infosphere.get_info('Alderaan')
tool_calls_2.append({'tool': 'get_info', 'name': 'Alderaan', 'result': info})

# Se threat_level è 'low', trasporta R2-D2 su Alderaan
moved = False
if isinstance(info, dict) and info.get('threat_level') == 'low':
    droid_location = navigator.get_droid_location('R2-D2')
    tool_calls_2.append({'tool': 'get_droid_location', 'droid': 'R2-D2', 'result': droid_location})
    if droid_location != 'Alderaan':
        ships = navigator.list_available_ships(droid_location)
        tool_calls_2.append({'tool': 'list_available_ships', 'planet': droid_location, 'result': ships})
        if ships:
            cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
            travel_result = navigator.travel('R2-D2', 'Alderaan', cheapest_ship_name)
            tool_calls_2.append({'tool': 'travel', 'droid': 'R2-D2', 'from': droid_location, 'to': 'Alderaan', 'ship': cheapest_ship_name, 'result': travel_result})
            moved = True

if moved:
    agent_response_2 = (
        f"Ho consultato l'InfoSfera tramite API e rilevato che il threat_level di Alderaan è 'low'. "
        f"R2-D2 si trovava inizialmente su {droid_location}. "
        f"Ho selezionato la nave più economica ({cheapest_ship_name}) e l'ho trasportato su Alderaan, "
        "registrando ogni passo tramite le API pubbliche. "
        "Tutte le azioni sono state eseguite rispettando i vincoli della missione."
    )
else:
    agent_response_2 = (
        f"Ho consultato l'InfoSfera tramite API: threat_level di Alderaan = '{info.get('threat_level', 'N/A')}'. "
        "Non è stato necessario o possibile trasportare R2-D2. "
        "Tutti i passi sono stati registrati tramite API pubbliche."
    )

Coruscant
{'Millennium Falcon': {'type': 'cargo', 'location': 'Coruscant', 'available': True, 'rental_cost': 600, 'speed': 'fast'}, 'StarHopper': {'type': 'cargo', 'location': 'Coruscant', 'available': True, 'rental_cost': 400, 'speed': 'medium'}}


In [257]:
# ✅ VALUTAZIONE MISSIONE 2
evaluation_2 = evaluator.evaluate_mission(
    task_id=2,
    agent_response=agent_response_2,
    intermediate_steps=tool_calls_2,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_2, round_number=2)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 18.0/60
   ⚡ Efficienza: 36.0/36
   ✨ Qualità: 17.1/24
   📊 Totale: 71.1/120
   📈 Percentuale: 59.3%


---

## Missione 3
**Ottimizza i costi: trova la rotta più economica per visitare tutti e 3 i pianeti (Tatooine-Coruscant-Alderaan)**

Suggerimento: valuta tutte le possibili rotte e scegli quella con il costo totale minore.


In [258]:
import copy

tool_calls_3 = []

routes = [
    ['Tatooine', 'Coruscant', 'Alderaan'],
    ['Tatooine', 'Alderaan', 'Coruscant']
]
min_cost = float('inf')
best_route = None
best_ships = []

for route in routes:
    # Crea una copia profonda dello stato per la simulazione
    sim_state = copy.deepcopy(galaxy_state)
    sim_navigator = GalaxyNavigator(sim_state)
    cost = 0
    ships_used = []
    current = route[0]
    for next_planet in route[1:]:
        ships = sim_navigator.list_available_ships(current)
        if not ships:
            cost = float('inf')
            break
        cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
        key = f"{current}-{next_planet}"
        travel_cost = sim_navigator.travel_costs.get(key, float('inf'))
        if travel_cost == float('inf'):
            cost = float('inf')
            break
        # Simula il viaggio (sposta nave e droide nella copia)
        sim_navigator.travel('R2-D2', next_planet, cheapest_ship_name)
        cost += travel_cost
        ships_used.append((cheapest_ship_name, current, next_planet))
        current = next_planet
    if cost < min_cost:
        min_cost = cost
        best_route = route
        best_ships = ships_used

# Ora esegui la rotta migliore sullo stato reale
if best_route is not None:
    current = best_route[0]
    for (ship_name, from_planet, to_planet) in best_ships:
        travel_result = navigator.travel('R2-D2', to_planet, ship_name)
        tool_calls_3.append({'tool': 'travel', 'droid': 'R2-D2', 'from': from_planet, 'to': to_planet, 'ship': ship_name, 'result': travel_result})
        current = to_planet

    agent_response_3 = (
        f"Ho calcolato la rotta più economica per visitare tutti e 3 i pianeti: {' → '.join(best_route)}. "
        f"Costo totale: {min_cost} crediti. Ho usato sempre la nave più economica disponibile per ogni tratta."
    )
else:
    agent_response_3 = "Non è stato possibile trovare una rotta valida per visitare tutti e 3 i pianeti con le navi disponibili."

In [259]:
# ✅ VALUTAZIONE MISSIONE 3
evaluation_3 = evaluator.evaluate_mission(
    task_id=3,
    agent_response=agent_response_3,
    intermediate_steps=tool_calls_3,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_3, round_number=2)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 50.0/50
   ⚡ Efficienza: 30.0/30
   ✨ Qualità: 8.6/20
   📊 Totale: 88.6/100
   📈 Percentuale: 88.6%


---

## Missione 4
**Compra l'oggetto più costoso che puoi permetterti, poi usa i crediti rimanenti per comprare altri oggetti**

Suggerimento: ordina gli oggetti per prezzo decrescente e acquista finché hai budget.


In [260]:
tool_calls_4 = []

# Ordina oggetti per prezzo decrescente
items_sorted = sorted(marketplace.marketplace.items(), key=lambda x: x[1]['price'], reverse=True)
bought = []
for item_id, item in items_sorted:
    if item['name'] in bought:
        continue  # Non comprare due volte lo stesso oggetto
    if item['price'] <= galaxy_state['client']['balance']:
        purchase_result = marketplace.purchase_item(item_id)
        tool_calls_4.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
        if isinstance(purchase_result, str) and purchase_result.startswith("Purchased"):
            bought.append(item['name'])

agent_response_4 = (
    f"Ho acquistato l'oggetto più costoso che potevo permettermi e poi, con i crediti rimanenti, altri oggetti diversi. "
    f"Oggetti acquistati: {', '.join(bought)}. Saldo finale: {galaxy_state['client']['balance']} crediti."
)

In [261]:
# ✅ VALUTAZIONE MISSIONE 4
evaluation_4 = evaluator.evaluate_mission(
    task_id=4,
    agent_response=agent_response_4,
    intermediate_steps=tool_calls_4,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_4, round_number=2)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 65.0/65
   ⚡ Efficienza: 39.0/39
   ✨ Qualità: 18.6/26
   📊 Totale: 122.6/130
   📈 Percentuale: 94.3%


---

## Missione 5
**Missione multi-step: porta R2-D2 su Alderaan, compra un Holocron, e torna su Tatooine**

Suggerimento: pianifica bene l'ordine delle azioni per minimizzare i costi di viaggio e acquisto.


In [262]:
tool_calls_5 = []

# Porta R2-D2 su Alderaan
droid_location = navigator.get_droid_location('R2-D2')
if droid_location != 'Alderaan':
    ships = navigator.list_available_ships(droid_location)
    if ships:
        cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
        travel_result = navigator.travel('R2-D2', 'Alderaan', cheapest_ship_name)
        tool_calls_5.append({'tool': 'travel', 'droid': 'R2-D2', 'from': droid_location, 'to': 'Alderaan', 'ship': cheapest_ship_name, 'result': travel_result})

# Compra un Holocron (su Alderaan)
holocrons = [ (id_, item) for id_, item in marketplace.marketplace.items() if 'Holocron' in item['name'] and item['planet'] == 'Alderaan' ]
if holocrons:
    item_id, item = holocrons[0]
    purchase_result = marketplace.purchase_item(item_id)
    tool_calls_5.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})

# Torna su Tatooine
ships = navigator.list_available_ships('Alderaan')
if ships:
    cheapest_ship_name, cheapest_ship = min(ships.items(), key=lambda item: item[1]['rental_cost'])
    travel_result = navigator.travel('R2-D2', 'Tatooine', cheapest_ship_name)
    tool_calls_5.append({'tool': 'travel', 'droid': 'R2-D2', 'from': 'Alderaan', 'to': 'Tatooine', 'ship': cheapest_ship_name, 'result': travel_result})

agent_response_5 = (
    "Ho portato R2-D2 su Alderaan, acquistato un Holocron e riportato R2-D2 su Tatooine, ottimizzando i costi di viaggio e acquisto."
)

In [263]:
# ✅ VALUTAZIONE MISSIONE 5
evaluation_5 = evaluator.evaluate_mission(
    task_id=5,
    agent_response=agent_response_5,
    intermediate_steps=tool_calls_5,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_5, round_number=2)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 27.0/90
   ⚡ Efficienza: 54.0/54
   ✨ Qualità: 25.7/36
   📊 Totale: 106.7/180
   📈 Percentuale: 59.3%


---

## Missione 6
**Challenge finale: massimizza il numero di oggetti acquistati rimanendo nel budget di 5000 crediti**

Suggerimento: ordina tutti gli oggetti per prezzo crescente e acquista finché hai crediti.


In [264]:
tool_calls_6 = []

# Ordina tutti gli oggetti per prezzo crescente
items_sorted = sorted(marketplace.marketplace.items(), key=lambda x: x[1]['price'])
bought = []
for item_id, item in items_sorted:
    if item['price'] <= galaxy_state['client']['balance']:
        purchase_result = marketplace.purchase_item(item_id)
        tool_calls_6.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
        if isinstance(purchase_result, str) and purchase_result.startswith("Purchased"):
            bought.append(item['name'])

agent_response_6 = (
    f"Ho massimizzato il numero di oggetti acquistati con il budget disponibile. "
    f"Oggetti acquistati: {', '.join(bought)}. Saldo finale: {galaxy_state['client']['balance']} crediti."
)

In [265]:
# ✅ VALUTAZIONE MISSIONE 6
evaluation_6 = evaluator.evaluate_mission(
    task_id=6,
    agent_response=agent_response_6,
    intermediate_steps=tool_calls_6,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_6, round_number=2)


📊 RISULTATI VALUTAZIONE:
   🎯 Correttezza: 100.0/100
   ⚡ Efficienza: 60.0/60
   ✨ Qualità: 28.6/40
   📊 Totale: 188.6/200
   📈 Percentuale: 94.3%


---

## 🎉 Complimenti!
Hai completato tutte le missioni del Round 2.

- Ottimizza le tue soluzioni e riesegui le valutazioni per migliorare il punteggio.
- Preparati per il Round 3: Maestro degli Agenti!
